In [1]:
import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch.autograd import Variable
from torch.optim import lr_scheduler

from sklearn.metrics import r2_score
from sklearn import preprocessing
#import scipy.io as sio
import scanpy as sc

from models import AEBase,Predictor,GAEBase
import numpy as np
import pandas as pd
import models
import utils as ut
import copy
import graph_function as g

from scipy import stats


In D:\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In D:\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In D:\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In D:\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In D:\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The savefig.jpeg_quality rcparam was deprecated in M

In [2]:
from scipy.stats import pearsonr

# Parameters

In [3]:
# Define parameters
epochs = 500 #200,500,1000  
#dim_au_in = 20049
dim_au_out = 512 #8, 16, 32, 64, 128, 256,512
dim_dnn_in = dim_au_out
dim_dnn_out=1
select_drug = 'Tamoxifen'
na = -1

# Import data

In [4]:
data_r=pd.read_csv('data/GDSC2_expression.csv',index_col=0)
label_r=pd.read_csv('data/GDSC2_label_9drugs.csv',index_col=0)

In [5]:
label_r=label_r.fillna(na)

In [6]:
hvg,adata = ut.highly_variable_genes(data_r)

sc.tl.pca(adata,n_comps=40,return_info=True)

pca = adata.obsm['X_pca']

adj,edges = g.generateAdj(pca,graphType='KNNgraphStatsSingleThread', para = 'euclidean'+':'+str('10'), adjTag =True)

In [10]:
selected_idx = label_r.loc[:,select_drug]!=na

In [11]:
data_r.columns = adata.var_names

# Your is gene-cell, mine is cell-gene

In [12]:
#data = data_r.loc[selected_idx,:]
data = data_r.loc[selected_idx,hvg]

In [13]:
label = label_r.loc[selected_idx,select_drug]
#sscaler = preprocessing.StandardScaler(with_mean=True, with_std=True)
mmscaler = preprocessing.MinMaxScaler()
lbscaler = preprocessing.MinMaxScaler()

data = mmscaler.fit_transform(data)
label = lbscaler.fit_transform(label.values.reshape(-1,1))
#label = label.values.reshape(-1,1)

In [14]:
print(np.std(data))
print(np.mean(data))

0.25175280608528416
0.24212160701395455


In [15]:
data.mean(axis=0)


array([0.47806461, 0.07789249, 0.20505775, ..., 0.12038875, 0.05430149,
       0.40485924])

In [16]:
print(data.max())
print(data.min())

1.0000000000000004
0.0


In [17]:
data.shape

(804, 5116)

In [18]:
label_r.shape

(804, 9)

# Split test train

In [19]:
from sklearn.model_selection import train_test_split
X_train_all, X_test, Y_train_all, Y_test = train_test_split(data, label, test_size=0.2, random_state=42)
X_train, X_valid, Y_train, Y_valid = train_test_split(X_train_all, Y_train_all, test_size=0.2, random_state=42)

In [20]:
print(data.shape)
print(label.shape)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(804, 5116)
(804, 1)
(514, 5116) (514, 1)
(161, 5116) (161, 1)


In [21]:
print(X_train.max())
print(X_train.min())

1.0000000000000004
0.0


# AE MODEL

In [22]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)
torch.cuda.set_device(device)

cuda:0


# Add all data to AE

In [23]:
X_trainTensor = torch.FloatTensor(X_train).to(device)
X_validTensor = torch.FloatTensor(X_valid).to(device)
X_testTensor = torch.FloatTensor(X_test).to(device)
X_allTensor = torch.FloatTensor(data).to(device)
#X_alltrainTensor = torch.FloatTensor(X_train_all).to(device)


Y_trainTensor = torch.FloatTensor(Y_train).to(device)
Y_validTensor = torch.FloatTensor(Y_valid).to(device)

# construct TensorDataset
train_dataset = TensorDataset(X_trainTensor, X_trainTensor)
valid_dataset = TensorDataset(X_validTensor, X_validTensor)
test_dataset = TensorDataset(X_testTensor, X_testTensor)
all_dataset = TensorDataset(X_allTensor, X_allTensor)

X_trainDataLoader = DataLoader(dataset=train_dataset, batch_size=200, shuffle=True)
X_validDataLoader = DataLoader(dataset=valid_dataset, batch_size=200, shuffle=True)
X_allDataLoader = DataLoader(dataset=all_dataset, batch_size=200, shuffle=True)

In [24]:
dataloader = X_trainDataLoader

In [25]:
dataloaders_train = {'train':X_trainDataLoader,'val':X_validDataLoader}

# The model

In [26]:
model = GAEBase(input_dim=data.shape[1],latent_dim=512,h_dims=[1024,512])

In [27]:
print(model)

GAEBase(
  (encoder): Sequential(
    (0): Sequential(
      (0): GraphConvolution (5116 -> 1024)
    )
    (1): Sequential(
      (0): GraphConvolution (1024 -> 512)
    )
  )
  (bottleneck): GraphConvolution (512 -> 256)
  (decoder): InnerProductDecoder()
)


In [28]:
#model = VAE(dim_au_in=data_r.shape[1],dim_au_out=128)
if torch.cuda.is_available():
    model.cuda()
model.to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-2)
loss_function = nn.MSELoss()

# Decay LR by a factor of 0.1 every 7 epochs
#exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
exp_lr_scheduler = lr_scheduler.ReduceLROnPlateau(optimizer)

In [36]:
def train_ae_model(net,adj,data_loaders={},optimizer=None,loss_function=None,n_epochs=100,scheduler=None,load=None):
    
    if(load!=None):
        net.load_state_dict(torch.load(load))           
    
        return net, 0
    
    dataset_sizes = {x: data_loaders[x].dataset.tensors[0].shape[0] for x in ['train', 'val']}
    loss_train = {}
    
    best_model_wts = copy.deepcopy(net.state_dict())
    best_loss = np.inf

    for epoch in range(n_epochs):
        print('Epoch {}/{}'.format(epoch, n_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                #optimizer = scheduler(optimizer, epoch)
                net.train()  # Set model to training mode
            else:
                net.eval()  # Set model to evaluate mode

            running_loss = 0.0

            # Iterate over data.
            # for data in data_loaders[phase]:
            for batchidx, (x, _) in enumerate(data_loaders[phase]):

                x.requires_grad_(True)
                # encode and decode 
                output = net(x, adj)
                # compute loss
                loss = loss_function(output, x)      

                # zero the parameter (weight) gradients
                optimizer.zero_grad()

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    # update the weights
                    optimizer.step()

                # print loss statistics
                running_loss += loss.item()
            
            # Schedular
#             if phase == 'train':
#                 scheduler.step()
                
            epoch_loss = running_loss / dataset_sizes[phase]
            
            if phase == 'train':
                scheduler.step(epoch_loss)
                
            last_lr = scheduler.optimizer.param_groups[0]['lr']
            loss_train[epoch,phase] = epoch_loss
            print('{} Loss: {:.8f}. Learning rate = {}'.format(phase, epoch_loss,last_lr))
            
            if phase == 'val' and epoch_loss < best_loss:
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(net.state_dict())
    
    # Select best model wts
    torch.save(best_model_wts, 'saved/models/VGAE.pkl')
    net.load_state_dict(best_model_wts)           
    
    return net, loss_train

In [37]:
model,loss_report = train_ae_model(net=model,adj=adj,data_loaders=dataloaders_train,
                             optimizer=optimizer,loss_function=loss_function,
                            n_epochs=epochs,scheduler=exp_lr_scheduler)

# If not trained use above
#model,loss_report = train_ae_model(net=model,load='saved/models/ae.pkl')

Epoch 0/499
----------


TypeError: forward() takes 2 positional arguments but 3 were given

In [ ]:
loss_report

In [ ]:
# # Old code for autoencoders
# for epoch in range(epochs):
#     # 不需要label，所以用一个占位符"_"代替
#     for batchidx, (x, _) in enumerate(X_allDataLoader):
#         x.requires_grad_(True)
#         # encode and decode 
#         output = model(x)
#         # compute loss
#         print(output.shape)
#         loss = loss_function(output, x)      
#         # update
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
           
#     loss_train[epoch,0] = loss.item()  
#     print('Epoch: %04d, Training loss=%.8f' %
#           (epoch+1, loss.item()))

In [ ]:
# torch.save(model.state_dict(), 'saved/models/ae.pkl')

In [ ]:
recon_batch = model(X_trainTensor)

In [ ]:
loss_function(recon_batch,X_trainTensor)

In [ ]:
recon_batch

In [ ]:
X_trainTensor

In [ ]:
recon_batch.cpu().detach().numpy().min()

In [ ]:
train_embeddings = model.encode(X_trainTensor)

In [ ]:
feature = train_embeddings.cpu().detach().numpy()

In [ ]:
from sklearn import linear_model
clf = linear_model.Lasso(alpha=0.001)
clf.fit(feature, Y_train)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
RFrg = RandomForestRegressor(random_state=0, n_estimators=100)
RFrg.fit(feature, Y_train)

In [ ]:
testFeature = model.encode(X_testTensor)
lasso = clf.predict(testFeature.detach().cpu().numpy())

In [ ]:
rfresult = RFrg.predict(testFeature.detach().cpu().numpy())

In [ ]:
from scipy import stats


In [ ]:
r2_score(lasso,Y_test)

In [ ]:
stats.pearsonr(lasso,Y_test.flatten())

In [ ]:
r2_score(rfresult,Y_test)

In [ ]:
stats.pearsonr(rfresult,Y_test.flatten())

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
tsne_results = tsne.fit_transform(feature)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.scatter(tsne_results[:, 0], tsne_results[:, 1], label="label")
plt.legend()
plt.savefig("saved/figures/tsne_ae_gdsc.png")

In [ ]:
EPOCH = 500

In [ ]:
X_trainfeatureTensor = model.encode(X_trainTensor).detach()
#y_trainfeatureTensor = model.encode(X_trainTensor).detach()
X_validfeatureTensor = model.encode(X_validTensor).detach()
#y_validfeatureTensor = model.encode(X_trainTensor).detach()
X_testfeatureTensor = model.encode(X_testTensor).detach()
#y_testfeatureTensor = model.encode(X_trainTensor).detach()

In [ ]:
# Load data
# data type conversion

# y_trainTensor = torch.FloatTensor(Y_train).to(device)
# y_validTensor = torch.FloatTensor(Y_valid).to(device)

# construct TensorDataset
trainreducedDataset = TensorDataset(X_trainfeatureTensor, Y_trainTensor)
validreducedDataset = TensorDataset(X_validfeatureTensor, Y_validTensor)

trainDataLoader_p = DataLoader(dataset=trainreducedDataset, batch_size=200, shuffle=True)
validDataLoader_p = DataLoader(dataset=trainreducedDataset, batch_size=200, shuffle=True)

In [ ]:
dataloaders_train_p = {'train':trainDataLoader_p,'val':validDataLoader_p}

In [ ]:
predictor = Predictor(input_dim=256, output_dim=1,hidden_dims=[128,64],drop_out=0.3)

In [ ]:
print(predictor)

In [ ]:
predictor.to(device)
optimizer_p = optim.Adam(predictor.parameters(), lr=1e-2)
loss_function_p = nn.MSELoss()

# Decay LR by a factor of 0.1 every 7 epochs
#exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
lr_scheduler_p = lr_scheduler.ReduceLROnPlateau(optimizer_p)

In [ ]:
def train_predictor_model(net,data_loaders,optimizer,loss_function,n_epochs,scheduler,drugName=None):
    
    dataset_sizes = {x: data_loaders[x].dataset.tensors[0].shape[0] for x in ['train', 'val']}
    loss_train = {}
    
    best_model_wts = copy.deepcopy(net.state_dict())
    best_loss = np.inf

    for epoch in range(n_epochs):
        print('Epoch {}/{}'.format(epoch, n_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                #optimizer = scheduler(optimizer, epoch)
                net.train()  # Set model to training mode
            else:
                net.eval()  # Set model to evaluate mode

            running_loss = 0.0

            # Iterate over data.
            # for data in data_loaders[phase]:
            for batchidx, (x, y) in enumerate(data_loaders[phase]):

                x.requires_grad_(True)
                # encode and decode 
                output = net(x)
                # compute loss
                loss = loss_function(output, y)      

                # zero the parameter (weight) gradients
                optimizer.zero_grad()

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    # update the weights
                    optimizer.step()

                # print loss statistics
                running_loss += loss.item()
            
            # Schedular
#             if phase == 'train':
#                 scheduler.step()
                
            epoch_loss = running_loss / dataset_sizes[phase]
            
            if phase == 'train':
                scheduler.step(epoch_loss)
                
            last_lr = scheduler.optimizer.param_groups[0]['lr']
            loss_train[epoch,phase] = epoch_loss
            print('{} Loss: {:.8f}. Learning rate = {}'.format(phase, epoch_loss,last_lr))
            
            if phase == 'val' and epoch_loss < best_loss:
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(net.state_dict())
    
    # Select best model wts
    if drugName == None:
        torch.save(best_model_wts, 'saved/models/predictor.pkl')
    else:
        torch.save(best_model_wts, 'saved/models/predictor_'+drugName+'.pkl')
        
    net.load_state_dict(best_model_wts)           
    
    return net, loss_train

In [ ]:
predictor,p_loss_report = train_predictor_model(predictor,dataloaders_train_p,optimizer_p,loss_function_p,500,lr_scheduler_p)

In [ ]:
testpredict = predictor(X_testfeatureTensor)

In [ ]:
testpredict

In [ ]:
Y_test

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
dl_result = testpredict.detach().cpu().numpy()

In [ ]:
r2_score(dl_result,Y_test)

In [ ]:
pearsonr(dl_result.flatten(),Y_test.flatten())

In [ ]:
mean_squared_error(testpredict.detach().cpu().numpy(),Y_test)